<H1> <center>Fragility Curves</center> </H1>

In [1]:
# Imports
import os
import math
import ctypes
from time import sleep
from tkinter import ttk
import tkinter as tk
import tkinter.font as tkFont
import tkinter.filedialog as fdlg
import numpy  as np
import matplotlib.pyplot as plt
#%matplotlib notebook
%matplotlib inline
from scipy.stats import norm
from scipy import fft, linspace
from scipy.interpolate import spline
import ipywidgets as widgets
from ipywidgets import interact
import plotly
#import plotly.plotly as py
#import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)         # initiate notebook for offline plot

## *Earthquake Recordings*
11 ground motion records.


| No. |        Location       |    Date    | Database Code | Station Code | Repi (km) |  Mw  | PGA (m/s2) | Vs,30 (m/s) | Soil type (EC8) |   Fault mech.   |
|:---:|:---------------------:|:----------:|:-------------:|:------------:|:---------:|:----:|:----------:|:-----------:|:---------------:|:---------------:|
|  1  |       Tabas/Iran      |  16/9/1978 |    ESMD_59    |      DAY     |   12.00   | 7.35 |    3.16    |    826.00   |        A        |     Oblique     |
|  2  | Montenegro/Montenegro |  15/4/1979 |   ISESD_223   |      ULA     |   21.00   | 6.90 |    1.77    |   1083.00   |        A        |      Thrust     |
|  3  |    App.Lucano/Italy   |  9/9/1998  |   ITACA_614   |      LRS     |    9.80   | 5.60 |    1.62    |   1024.00   |        A        |      Normal     |
|  4  |       Kobe/Japan      |  16/1/1995 |    NGA_1108   |      KBU     |   25.40   | 6.90 |    2.85    |   1043.00   |        A        |   Strike-Slip   |
|  5  |  Sierra Madre/Mexico  |  28/6/1991 |    NGA_1645   |      MTW     |    6.46   | 5.61 |    2.71    |    821.69   |        A        |     Reverse     |
|  6  |    Loma Prieta/USA    | 18/10/1989 |    NGA_3548   |      LEX     |   20.35   | 6.93 |    4.12    |   1070.34   |        A        | Reverse-Oblique |
|  7  |  Whittier Narrows/USA |  1/10/1987 |    NGA_680    |      KRE     |   13.85   | 5.99 |    1.10    |    969.07   |        A        | Reverse-Oblique |
|  8  |     Northridge/USA    |  17/1/1994 |    NGA_994    |      GPO     |   25.42   | 6.69 |    2.84    |   1015.88   |        A        |     Reverse     |
|  9  |      Izmit/Turkey     |  17/8/1999 |  T-NSMP_1109  |     4101     |    3.40   | 7.60 |    1.65    |    826.11   |        A        |   Strike-Slip   |
|  10 |   East Sicily/Italy   | 13/12/1990 |   ITACA_314   |      SRT     |   28.30   | 5.60 |    0.61    |    871.00   |        A        |   Strike-Slip   |
|  11 | Western Tottori/Japan |  6/10/2000 |  KIK-Net_3775 |    SMNH10    |   31.37   | 6.60 |    1.55    |    967.27   |        A        |   Strike-Slip   |

In [2]:
# Load Ground Motions
GM={}
# Details
# Location
GM['Location']={}
GM['Location'][1]='Tabas/Iran'
GM['Location'][2]='Montenegro/Montenegro'
GM['Location'][3]='App.Lucano/Italy'
GM['Location'][4]='Kobe/Japan'
GM['Location'][5]='Sierra Madre/Mexico'
GM['Location'][6]='Loma Prieta/USA'
GM['Location'][7]='Whittier Narrows/USA'
GM['Location'][8]='Northridge/USA'
GM['Location'][9]='Izmit/Turkey'
GM['Location'][10]='East Sicily/Italy'
GM['Location'][11]='Western Tottori/Japan'
# Station
GM['Station']={}
GM['Station'][1]='DAY'
GM['Station'][2]='ULA'
GM['Station'][3]='LRS'
GM['Station'][4]='KBU'
GM['Station'][5]='MTW'
GM['Station'][6]='LEX'
GM['Station'][7]='KRE'
GM['Station'][8]='GPO'
GM['Station'][9]='4101'
GM['Station'][10]='SRT'
GM['Station'][11]='SMNH10'
# Epicentral Distance [km]
GM['EpicDist']={}
GM['EpicDist'][1]=12.00
GM['EpicDist'][2]=21.00
GM['EpicDist'][3]=9.80
GM['EpicDist'][4]=25.40
GM['EpicDist'][5]=6.46
GM['EpicDist'][6]=20.35
GM['EpicDist'][7]=13.85
GM['EpicDist'][8]=25.42
GM['EpicDist'][9]=3.40
GM['EpicDist'][10]=28.30
GM['EpicDist'][11]=31.37
# Magnitude Mw
GM['Mw']={}
GM['Mw'][1]=7.35
GM['Mw'][2]=6.90
GM['Mw'][3]=5.60
GM['Mw'][4]=6.90
GM['Mw'][5]=5.61
GM['Mw'][6]=6.93
GM['Mw'][7]=5.99
GM['Mw'][8]=6.69
GM['Mw'][9]=7.60
GM['Mw'][10]=5.60
GM['Mw'][11]=6.60
# Vs at site [m/s]
GM['Vs']={}
GM['Vs'][1]=826.00
GM['Vs'][2]=1083.00
GM['Vs'][3]=1024.00
GM['Vs'][4]=1043.00
GM['Vs'][5]=821.69
GM['Vs'][6]=1070.34
GM['Vs'][7]=969.07
GM['Vs'][8]=1015.88
GM['Vs'][9]=826.11
GM['Vs'][10]=871.00
GM['Vs'][11]=967.27
# EC8 soil type at site
GM['EC8soil']={}
GM['EC8soil'][1]='A'
GM['EC8soil'][2]='A'
GM['EC8soil'][3]='A'
GM['EC8soil'][4]='A'
GM['EC8soil'][5]='A'
GM['EC8soil'][6]='A'
GM['EC8soil'][7]='A'
GM['EC8soil'][8]='A'
GM['EC8soil'][9]='A'
GM['EC8soil'][10]='A'
GM['EC8soil'][11]='A'
# Fault mechanism (RV: reverse; OB: oblique; SS: strike-slip; NM: normal; TH: thrust)
GM['FaultMech']={}
GM['FaultMech'][1]='OB'
GM['FaultMech'][2]='TH'
GM['FaultMech'][3]='NM'
GM['FaultMech'][4]='SS'
GM['FaultMech'][5]='RV'
GM['FaultMech'][6]='RV-OB'
GM['FaultMech'][7]='RV-OB'
GM['FaultMech'][8]='RV'
GM['FaultMech'][9]='SS'
GM['FaultMech'][10]='SS'
GM['FaultMech'][11]='SS'
# Acceleration
GM['Acc']={}
GM['Acc'][1]='EQinp/Acc/ESMD_59_H1_acc.txt'
GM['Acc'][2]='EQinp/Acc/ISESD_223_H1_acc.txt'
GM['Acc'][3]='EQinp/Acc/ITACA_614_H1_acc.txt'
GM['Acc'][4]='EQinp/Acc/NGA_1108_H1_acc.txt'
GM['Acc'][5]='EQinp/Acc/NGA_1645_H1_acc.txt'
GM['Acc'][6]='EQinp/Acc/NGA_3548_H1_acc.txt'
GM['Acc'][7]='EQinp/Acc/NGA_680_H1_acc.txt'
GM['Acc'][8]='EQinp/Acc/NGA_994_H1_acc.txt'
GM['Acc'][9]='EQinp/Acc/T-NSMP_1109_H1_acc.txt'
GM['Acc'][10]='EQinp/Acc/ITACA_314_H1_acc.txt'
GM['Acc'][11]='EQinp/Acc/KIK-Net_3775_H1_acc.txt'
# Velocity
GM['Vel']={}
GM['Vel'][1]='EQinp/Vel/ESMD_59_H1_vel.txt'
GM['Vel'][2]='EQinp/Vel/ISESD_223_H1_vel.txt'
GM['Vel'][3]='EQinp/Vel/ITACA_614_H1_vel.txt'
GM['Vel'][4]='EQinp/Vel/NGA_1108_H1_vel.txt'
GM['Vel'][5]='EQinp/Vel/NGA_1645_H1_vel.txt'
GM['Vel'][6]='EQinp/Vel/NGA_3548_H1_vel.txt'
GM['Vel'][7]='EQinp/Vel/NGA_680_H1_vel.txt'
GM['Vel'][8]='EQinp/Vel/NGA_994_H1_vel.txt'
GM['Vel'][9]='EQinp/Vel/T-NSMP_1109_H1_vel.txt'
GM['Vel'][10]='EQinp/Vel/ITACA_314_H1_vel.txt'
GM['Vel'][11]='EQinp/Vel/KIK-Net_3775_H1_vel.txt'

PGA_EQ={}
Len_EQ={}
EQ_con={}
for i in GM['Acc']:
    EQ_temp=[]
    with open(GM['Acc'][i],'r') as f:
        for line in f:
            string_list = line.split()
            for field in string_list:
                EQ_temp.append(float(field))
    f.close()
    EQ_con[i]=EQ_temp[1::2]

In [4]:

# Plot Ground Motions
def GMplot(i):
    t = np.arange(0.0,len(EQ_con[i]),1)
    t=t/100.0
    trace = Scatter(
    x = t,
    y = EQ_con[i]
    );
    data = [trace];
    iplot(data, filename='basic-line');
    return
interact(GMplot, i=widgets.Dropdown(options={GM['Location'][1]: 1,GM['Location'][2]: 2,GM['Location'][3]: 3,GM['Location'][4]: 4,GM['Location'][5]: 5,
                   GM['Location'][6]: 6,GM['Location'][7]: 7,GM['Location'][8]: 8,GM['Location'][9]: 9,GM['Location'][10]: 10,
                   GM['Location'][11]: 11},description='Ground Motion:',style=dict(description_width='30%')));

A Jupyter Widget

## *Response Spectra*


## *Fragility Curves*
